# Recommender System Cosine Final V1 #

### import necessary libraries ###

In [21]:
import pymysql
import pandas as pd

# jaccard similarity 
from scipy.spatial.distance import pdist, squareform

# 引入套件計算以餘弦相似度
from sklearn.metrics.pairwise import cosine_similarity

### Making a sql connection to database ###

In [22]:
conn = pymysql.connect(host = 'cgi101-db.ckelxbpeajyc.ap-northeast-3.rds.amazonaws.com',
                                    database='project_v5',
                                    user='',
                                    password='')

### Create a cursor ###

In [23]:
cursor = conn.cursor()

### Execute SQL ###

cursor.execute("""SELECT
	concat(dname,tpname) drink_name, dname,typename, dprice,tpname AS topping,  tpprice,  (dprice+tpprice) AS total 
FROM 
	(SELECT dname,dprice,typename FROM project_v5.dmenu 
			where dprice > 1  
			AND dname NOT LIKE '%元' and dname NOT LIKE '%特價%' and typename NOT LIKE '%外送%') d
		CROSS JOIN
	(SELECT tpname, tpprice FROM project_v5.toppings where tpprice > 1
			AND tpno < '000037' and tpno not in ('000024','000026','000027'))t  order by drink_name;""")

In [24]:
cursor.execute("""SELECT
	concat(dname,tpname) drink_name, dname,typename, dprice,tpname AS topping,  tpprice,  (dprice+tpprice) AS total 
FROM 
	(SELECT tt.d_name dname,dprice, tt.typename
FROM (SELECT if(INSTR(t.dname,'(XL)'),SUBSTR(t.dname,1,INSTR(t.dname,'(XL)')-1),t.dname) d_name, dprice,
	if(INSTR(t.typename,'(XL)'),SUBSTR(t.typename,1,INSTR(t.typename,'(XL)')-1),t.typename) typename
FROM (SELECT if(INSTR(dname,'(L)'),SUBSTR(dname,1,INSTR(dname,'(L)')-1),dname) dname, dprice,
if(INSTR(typename,'(L)'),SUBSTR(typename,1,INSTR(typename,'(L)')-1),typename) typename FROM project_v5.dmenu ) t
) tt where tt.d_name NOT LIKE '%元%' AND tt.d_name NOT IN('             ※','贈品','購物袋') AND TYPENAME != '外送平台') d
		CROSS JOIN
	(SELECT tpname, tpprice FROM project_v5.toppings where tpprice > 1
			AND tpno < '000037' and tpno not in ('000024','000026','000027'))t  order by drink_name;
""")

2184

### Creating DataFrame

In [25]:
# get column names
fields = [field_md[0] for field_md in cursor.description]
# zip column names and rows
result = [dict(zip(fields,row)) for row in cursor.fetchall()]
df = pd.DataFrame(result)

In [26]:
df =  df.iloc[0::2,:].reset_index(drop=True)

In [27]:
df

,drink_name,dname,typename,dprice,topping,tpprice,total
0,仙草凍奶茶+仙草凍,仙草凍奶茶,奶茶,40,+仙草凍,5,45
1,仙草凍奶茶+咖啡凍,仙草凍奶茶,奶茶,50,+咖啡凍,5,55
2,仙草凍奶茶+布丁,仙草凍奶茶,奶茶,50,+布丁,5,55
3,仙草凍奶茶+椰果,仙草凍奶茶,奶茶,50,+椰果,5,55
4,仙草凍奶茶+檸檬,仙草凍奶茶,奶茶,40,+檸檬,10,50
...,...,...,...,...,...,...,...
1087,鮮葡萄柚綠咖啡凍*2,鮮葡萄柚綠,特調,40,咖啡凍*2,10,50
1088,鮮葡萄柚綠布丁*2,鮮葡萄柚綠,特調,40,布丁*2,10,50
1089,鮮葡萄柚綠椰果*2,鮮葡萄柚綠,特調,40,椰果*2,10,50
1090,鮮葡萄柚綠珍珠*2,鮮葡萄柚綠,特調,40,珍珠*2,10,50


### Load the ingredient data

In [28]:
ingredient = pd.read_csv("data_from_shop.csv")
ingredient

,dname,無糖甜度(0~10),總甜度,熱品/冷品,主要成分,次要成分1,次要成分2,次要成分3,水果/成分,溫du
0,仙草凍奶茶,2,3,熱/冷,奶茶,NaN,仙草凍,NaN,NaN,偏冷
1,仙草甘茶,0,1,熱/冷,NaN,仙草茶,NaN,NaN,NaN,偏冷
2,仙草蜜,10,3,冷,蜜茶,NaN,仙草凍,NaN,NaN,偏冷
3,仙草鮮奶凍,5,2,熱/冷,NaN,鮮奶,仙草凍,NaN,NaN,偏冷
4,冬瓜仙草茶,3,2,熱/冷,冬瓜茶,仙草茶,NaN,NaN,NaN,偏冷
...,...,...,...,...,...,...,...,...,...,...
70,鮮奶烏龍,0,2,熱/冷,烏龍茶,鮮奶,NaN,NaN,NaN,偏冷
71,鮮奶紅茶,0,2,熱/冷,紅茶,鮮奶,NaN,NaN,NaN,偏冷
72,鮮奶綠茶,0,2,熱/冷,綠茶,鮮奶,NaN,NaN,NaN,偏冷
73,鮮奶青茶,0,2,熱/冷,青茶,鮮奶,NaN,NaN,NaN,偏冷


### Merge two dfs (inner join)

In [29]:
df_final = pd.merge(df,ingredient, how='inner', on='dname')
df_final

,drink_name,dname,typename,dprice,topping,tpprice,total,無糖甜度(0~10),總甜度,熱品/冷品,主要成分,次要成分1,次要成分2,次要成分3,水果/成分,溫du
0,仙草凍奶茶+仙草凍,仙草凍奶茶,奶茶,40,+仙草凍,5,45,2,3,熱/冷,奶茶,NaN,仙草凍,NaN,NaN,偏冷
1,仙草凍奶茶+咖啡凍,仙草凍奶茶,奶茶,50,+咖啡凍,5,55,2,3,熱/冷,奶茶,NaN,仙草凍,NaN,NaN,偏冷
2,仙草凍奶茶+布丁,仙草凍奶茶,奶茶,50,+布丁,5,55,2,3,熱/冷,奶茶,NaN,仙草凍,NaN,NaN,偏冷
3,仙草凍奶茶+椰果,仙草凍奶茶,奶茶,50,+椰果,5,55,2,3,熱/冷,奶茶,NaN,仙草凍,NaN,NaN,偏冷
4,仙草凍奶茶+檸檬,仙草凍奶茶,奶茶,40,+檸檬,10,50,2,3,熱/冷,奶茶,NaN,仙草凍,NaN,NaN,偏冷
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,鮮葡萄柚綠咖啡凍*2,鮮葡萄柚綠,特調,40,咖啡凍*2,10,50,0,1,冷,綠茶,葡萄柚汁,NaN,NaN,水果類,偏冷
1088,鮮葡萄柚綠布丁*2,鮮葡萄柚綠,特調,40,布丁*2,10,50,0,1,冷,綠茶,葡萄柚汁,NaN,NaN,水果類,偏冷
1089,鮮葡萄柚綠椰果*2,鮮葡萄柚綠,特調,40,椰果*2,10,50,0,1,冷,綠茶,葡萄柚汁,NaN,NaN,水果類,偏冷
1090,鮮葡萄柚綠珍珠*2,鮮葡萄柚綠,特調,40,珍珠*2,10,50,0,1,冷,綠茶,葡萄柚汁,NaN,NaN,水果類,偏冷


### Checking Null Values

In [30]:
df_final.columns[1:]


Index(['dname', 'typename', 'dprice', 'topping', 'tpprice', 'total',
       '無糖甜度(0~10)', '總甜度', '熱品/冷品', '主要成分', '次要成分1', '次要成分2', '次要成分3',
       '水果/成分', '溫du'],
      dtype='object')

### One-hot Encoding Using pd.get_dummies

In [31]:
encoded_df = pd.get_dummies(df_final,columns=df_final.columns[1:])

### Set drink_name as Index

In [32]:
encoded_df.set_index("drink_name",inplace=True)
encoded_df

,dname_仙草凍奶茶,dname_仙草甘茶,dname_仙草蜜,dname_仙草鮮奶凍,dname_冬瓜仙草茶,dname_冬瓜檸檬,dname_冬瓜茶,dname_冬瓜調茶,dname_咖啡凍奶茶,dname_奶香青茶,...,次要成分2_金桔,次要成分3_梅子,次要成分3_百香顆粒,次要成分3_薄荷糖漿,次要成分3_蜂蜜,次要成分3_養樂多,水果/成分_可可粉,水果/成分_水果類,溫du_偏冷,溫du_偏熱
drink_name,,,,,,,,,,,,,,,,,,,,,
仙草凍奶茶+仙草凍,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
仙草凍奶茶+咖啡凍,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
仙草凍奶茶+布丁,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
仙草凍奶茶+椰果,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
仙草凍奶茶+檸檬,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
鮮葡萄柚綠咖啡凍*2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
鮮葡萄柚綠布丁*2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
鮮葡萄柚綠椰果*2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


### Cosine Similarity

In [33]:
# 計算餘弦相似度，並會以 np.array 形式輸出儲存
cosine_similarity_array = cosine_similarity(encoded_df)

# 將資料轉換為 DF 型態以方便進行資料檢視與處理
cosine_similarity_df = pd.DataFrame(cosine_similarity_array,
                                    index=encoded_df.index,
                                    columns=encoded_df.index)

In [34]:
cosine_similarity_df

drink_name,仙草凍奶茶+仙草凍,仙草凍奶茶+咖啡凍,仙草凍奶茶+布丁,仙草凍奶茶+椰果,仙草凍奶茶+檸檬,仙草凍奶茶+珍珠,仙草凍奶茶+蘆薈,仙草凍奶茶仙草凍*2,仙草凍奶茶原料加量,仙草凍奶茶咖啡凍*2,...,鮮葡萄柚綠+檸檬,鮮葡萄柚綠+珍珠,鮮葡萄柚綠+蘆薈,鮮葡萄柚綠仙草凍*2,鮮葡萄柚綠原料加量,鮮葡萄柚綠咖啡凍*2,鮮葡萄柚綠布丁*2,鮮葡萄柚綠椰果*2,鮮葡萄柚綠珍珠*2,鮮葡萄柚綠蘆薈*2
drink_name,,,,,,,,,,,,,,,,,,,,,
仙草凍奶茶+仙草凍,1.000000,0.750000,0.750000,0.750000,0.750000,0.916667,0.750000,0.666667,0.666667,0.750000,...,0.080064,0.160128,0.080064,0.160128,0.160128,0.160128,0.160128,0.160128,0.160128,0.160128
仙草凍奶茶+咖啡凍,0.750000,1.000000,0.916667,0.916667,0.666667,0.750000,0.666667,0.750000,0.750000,0.666667,...,0.160128,0.320256,0.160128,0.080064,0.080064,0.080064,0.080064,0.080064,0.080064,0.080064
仙草凍奶茶+布丁,0.750000,0.916667,1.000000,0.916667,0.666667,0.750000,0.666667,0.750000,0.750000,0.666667,...,0.160128,0.320256,0.160128,0.080064,0.080064,0.080064,0.080064,0.080064,0.080064,0.080064
仙草凍奶茶+椰果,0.750000,0.916667,0.916667,1.000000,0.666667,0.750000,0.666667,0.750000,0.750000,0.666667,...,0.160128,0.320256,0.160128,0.080064,0.080064,0.080064,0.080064,0.080064,0.080064,0.080064
仙草凍奶茶+檸檬,0.750000,0.666667,0.666667,0.666667,1.000000,0.750000,0.916667,0.750000,0.750000,0.916667,...,0.240192,0.080064,0.160128,0.320256,0.320256,0.320256,0.320256,0.320256,0.320256,0.160128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
鮮葡萄柚綠咖啡凍*2,0.160128,0.080064,0.080064,0.080064,0.320256,0.160128,0.320256,0.160128,0.160128,0.400320,...,0.769231,0.692308,0.769231,0.923077,0.923077,1.000000,0.923077,0.923077,0.923077,0.769231
鮮葡萄柚綠布丁*2,0.160128,0.080064,0.080064,0.080064,0.320256,0.160128,0.320256,0.160128,0.160128,0.320256,...,0.769231,0.692308,0.769231,0.923077,0.923077,0.923077,1.000000,0.923077,0.923077,0.769231
鮮葡萄柚綠椰果*2,0.160128,0.080064,0.080064,0.080064,0.320256,0.160128,0.320256,0.160128,0.160128,0.320256,...,0.769231,0.692308,0.769231,0.923077,0.923077,0.923077,0.923077,1.000000,0.923077,0.769231


### '仙草凍奶茶(L)+仙草凍'相似的前20個飲料

In [35]:
cosine_similarity_df.loc['仙草凍奶茶+仙草凍'].sort_values(ascending=False).head(20)

drink_name
仙草凍奶茶+仙草凍     1.000000
仙草凍奶茶+珍珠      0.916667
布丁奶茶+仙草凍      0.833333
咖啡凍奶茶+仙草凍     0.833333
仙草凍奶茶椰果*2     0.750000
仙草凍奶茶+咖啡凍     0.750000
椰果奶茶+仙草凍      0.750000
咖啡凍奶茶+布丁      0.750000
咖啡凍奶茶+椰果      0.750000
仙草凍奶茶蘆薈*2     0.750000
仙草凍奶茶咖啡凍*2    0.750000
仙草凍奶茶+蘆薈      0.750000
仙草凍奶茶+檸檬      0.750000
仙草凍奶茶+椰果      0.750000
仙草凍奶茶+布丁      0.750000
咖啡凍奶茶+咖啡凍     0.750000
仙草凍奶茶布丁*2     0.666667
仙草凍奶茶珍珠*2     0.666667
仙草凍奶茶原料加量     0.666667
仙草凍奶茶仙草凍*2    0.666667
Name: 仙草凍奶茶+仙草凍, dtype: float64

In [36]:
def top_k_items(similarity_df,item_name,k):
    return similarity_df.loc[item_name].sort_values(ascending=False).head(k+1).index[1:].to_list()

In [37]:
print(top_k_items(cosine_similarity_df,'仙草凍奶茶+仙草凍',5))

['仙草凍奶茶+珍珠', '布丁奶茶+仙草凍', '咖啡凍奶茶+仙草凍', '仙草凍奶茶椰果*2', '仙草凍奶茶+咖啡凍']


In [38]:
print(top_k_items(cosine_similarity_df,'鮮葡萄柚綠布丁*2',5))

['鮮葡萄柚綠珍珠*2', '鮮葡萄柚綠椰果*2', '鮮葡萄柚綠咖啡凍*2', '鮮葡萄柚綠原料加量', '鮮葡萄柚綠仙草凍*2']


In [39]:
print(top_k_items(cosine_similarity_df,'椰果奶茶+仙草凍',5))

['椰果奶茶+咖啡凍', '椰果奶茶+珍珠', '布丁奶茶+仙草凍', '珍珠奶茶+珍珠', '咖啡凍奶茶+仙草凍']


In [40]:
cosine_similarity_df.to_csv("drinks_similarity_toppings.csv")

In [41]:
df = pd.read_csv("drinks_similarity_toppings.csv",index_col=0)

In [42]:
df

,仙草凍奶茶+仙草凍,仙草凍奶茶+咖啡凍,仙草凍奶茶+布丁,仙草凍奶茶+椰果,仙草凍奶茶+檸檬,仙草凍奶茶+珍珠,仙草凍奶茶+蘆薈,仙草凍奶茶仙草凍*2,仙草凍奶茶原料加量,仙草凍奶茶咖啡凍*2,...,鮮葡萄柚綠+檸檬,鮮葡萄柚綠+珍珠,鮮葡萄柚綠+蘆薈,鮮葡萄柚綠仙草凍*2,鮮葡萄柚綠原料加量,鮮葡萄柚綠咖啡凍*2,鮮葡萄柚綠布丁*2,鮮葡萄柚綠椰果*2,鮮葡萄柚綠珍珠*2,鮮葡萄柚綠蘆薈*2
drink_name,,,,,,,,,,,,,,,,,,,,,
仙草凍奶茶+仙草凍,1.000000,0.750000,0.750000,0.750000,0.750000,0.916667,0.750000,0.666667,0.666667,0.750000,...,0.080064,0.160128,0.080064,0.160128,0.160128,0.160128,0.160128,0.160128,0.160128,0.160128
仙草凍奶茶+咖啡凍,0.750000,1.000000,0.916667,0.916667,0.666667,0.750000,0.666667,0.750000,0.750000,0.666667,...,0.160128,0.320256,0.160128,0.080064,0.080064,0.080064,0.080064,0.080064,0.080064,0.080064
仙草凍奶茶+布丁,0.750000,0.916667,1.000000,0.916667,0.666667,0.750000,0.666667,0.750000,0.750000,0.666667,...,0.160128,0.320256,0.160128,0.080064,0.080064,0.080064,0.080064,0.080064,0.080064,0.080064
仙草凍奶茶+椰果,0.750000,0.916667,0.916667,1.000000,0.666667,0.750000,0.666667,0.750000,0.750000,0.666667,...,0.160128,0.320256,0.160128,0.080064,0.080064,0.080064,0.080064,0.080064,0.080064,0.080064
仙草凍奶茶+檸檬,0.750000,0.666667,0.666667,0.666667,1.000000,0.750000,0.916667,0.750000,0.750000,0.916667,...,0.240192,0.080064,0.160128,0.320256,0.320256,0.320256,0.320256,0.320256,0.320256,0.160128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
鮮葡萄柚綠咖啡凍*2,0.160128,0.080064,0.080064,0.080064,0.320256,0.160128,0.320256,0.160128,0.160128,0.400320,...,0.769231,0.692308,0.769231,0.923077,0.923077,1.000000,0.923077,0.923077,0.923077,0.769231
鮮葡萄柚綠布丁*2,0.160128,0.080064,0.080064,0.080064,0.320256,0.160128,0.320256,0.160128,0.160128,0.320256,...,0.769231,0.692308,0.769231,0.923077,0.923077,0.923077,1.000000,0.923077,0.923077,0.769231
鮮葡萄柚綠椰果*2,0.160128,0.080064,0.080064,0.080064,0.320256,0.160128,0.320256,0.160128,0.160128,0.320256,...,0.769231,0.692308,0.769231,0.923077,0.923077,0.923077,0.923077,1.000000,0.923077,0.769231
